In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import *
from sklearn.metrics import roc_curve
from keras.callbacks import EarlyStopping
from keras import backend
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_keras import KerasModelWrapper
import matplotlib as mpl
from keras.models import load_model

DATASET_NAME = 'cifar10'
(x_train, y_train, x_test, y_test) = get_dataset(DATASET_NAME)
earlystop = EarlyStopping(monitor='val_loss', patience=0, restore_best_weights=True)

In [ ]:
def pixel2phase_before(images):
    img_fft = np.fft.fft2(images)
    phase = np.angle(img_fft)
    return phase

In [ ]:
## Phase before Permutation + MLP   (seed 87)
SECRET_SEED = 87

x_train_phase = pixel2phase_before(x_train)
x_test_phase = pixel2phase_before(x_test)
x_train_shuffle = permute_pixels(x_train_phase, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test_phase, SECRET_SEED)

order_model = get_ppd_model(name=DATASET_NAME, ppd=False)
order_model.summary()
order_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

order_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[earlystop])



In [ ]:
sess =  backend.get_session()
backend.set_learning_phase(False)
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

x_adv_order = get_adv_exapmles(order_model, x_test, sess, attack_params)


In [ ]:
def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)

x_test_phase = pixel2phase_before(x_test)
x_adv_phase = pixel2phase_before(x_adv_order)
x_shuffle = permute_pixels(x_test_phase, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_phase, SECRET_SEED)

pred, acc = test_model(order_model, x_shuffle, y_test)
print ('Order changed PPD MLP acc normally', acc)
pred, acc = test_model(order_model, x_adv_shuffle, y_test)
print ('Order changed PPD MLP acc adv', acc)


## Apply pixel2phase in hidden

In [ ]:
## Normal PPD MLP   (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

normal_model = get_ppd_model(name=DATASET_NAME, ppd=True)
normal_model.summary()
normal_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

normal_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[earlystop])



In [ ]:
def get_ppd_model_hiddenperm(name='mnist', ppd=True, layerno=1):
    if name == 'mnist':
        input_shape = (28, 28, 1)
    else:
        input_shape = (32, 32, 3)  # cifar10

    def pixel2phase(images):
        img_fft = tf.fft(tf.cast(images, tf.complex64))
        phase = tf.angle(img_fft)
        return phase
    
    input_tensor = Input(name='input_images', shape=input_shape, dtype='float32')
    
    
    inner = Flatten(name='flatten')(input_tensor)
    inner = Dense(800,  name='dense800', kernel_initializer='he_normal', activation='relu')(inner)
    
    if ppd and layerno==1:
        inner = Lambda(pixel2phase, name='pixel2phase')(inner)
        #inner = Flatten(name='flatten')(inner)
        
    inner = Dense(300,  name='dense300', kernel_initializer='he_normal', activation='relu')(inner)
    
    if ppd and layerno==2:
        inner = Lambda(pixel2phase, name='pixel2phase')(inner)
    
    y_pred = Dense(10,  name='dense10', kernel_initializer='he_normal', activation='softmax')(inner)

    return Model(inputs=[input_tensor], outputs=y_pred)


In [ ]:
## Phase in layer 1   (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

hidden1_model = get_ppd_model_hiddenperm(name=DATASET_NAME, ppd=True, layerno=2)
hidden1_model.summary()
hidden1_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hidden1_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[earlystop])



In [ ]:
## Phase in layer 2    (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

hidden2_model = get_ppd_model_hiddenperm(name=DATASET_NAME, ppd=True, layerno=2)
hidden2_model.summary()
hidden2_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hidden2_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[earlystop])



In [ ]:
sess =  backend.get_session()
backend.set_learning_phase(False)
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

x_adv_normal = get_adv_exapmles(normal_model, x_test, sess, attack_params)
x_adv_hid1 = get_adv_exapmles(hidden1_model, x_test, sess, attack_params)
x_adv_hid2 = get_adv_exapmles(hidden2_model, x_test, sess, attack_params)


In [ ]:
def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)


x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_n = permute_pixels(x_adv_normal, SECRET_SEED)

pred, acc = test_model(normal_model, x_shuffle, y_test)
print ('Normal PPD MLP acc normally', acc)
pred, acc = test_model(normal_model, x_adv_shuffle_n, y_test)
print ('Normal PPD MLP acc adv', acc)




x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_1 = permute_pixels(x_adv_hid1, SECRET_SEED)

pred, acc = test_model(hidden1_model, x_shuffle, y_test)
print ('Phase in layer 1 PPD MLP acc normally', acc)
pred, acc = test_model(hidden1_model, x_adv_shuffle_1, y_test)
print ('Phase in layer 1 PPD MLP acc adv', acc)


x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_2 = permute_pixels(x_adv_hid2, SECRET_SEED)

pred, acc = test_model(hidden2_model, x_shuffle, y_test)
print ('Phase in layer 2 PPD MLP acc normally', acc)
pred, acc = test_model(hidden2_model, x_adv_shuffle_2, y_test)
print ('Phase in layer 2 PPD MLP acc adv', acc)



## In CNNs: Apply pixel2phase in hidden
For mnist: 0 <= layerno <= 3

For cifar: 0 <= layerno <= 5

layerno=0 is the default ppd + cnn

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import *
from sklearn.metrics import roc_curve
from keras.callbacks import EarlyStopping
from keras import backend
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_keras import KerasModelWrapper
import matplotlib as mpl
from keras.models import load_model

DATASET_NAME = 'cifar10'
(x_train, y_train, x_test, y_test) = get_dataset(DATASET_NAME) 
earlystop = EarlyStopping(monitor='val_loss', patience=0, restore_best_weights=True)
BATCH_SIZE = 128
EPOCHS = 50

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


TypeError: __init__() got an unexpected keyword argument 'restore_best_weights'

In [ ]:
## JUST CNN

cnn_model = get_cnn_model(name=DATASET_NAME, ppd=False)
cnn_model.summary()
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.fit(x_train, y_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE, validation_data=(x_test, y_test), callbacks=[earlystop])
cnn_model.save('cnn_model_'+DATASET_NAME+'.hdf5')


In [ ]:
## CNN + PPD, SEED 87

SECRET_SEED = 87
#x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
#x_test_shuffle = permute_pixels(x_test, SECRET_SEED)
x_train_shuffle = x_train
x_test_shuffle = x_test

ppd_cnn_model = get_cnn_model(name=DATASET_NAME, ppd=True)
ppd_cnn_model.summary()
ppd_cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ppd_cnn_model.fit(x_train_shuffle, y_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE, validation_data=(x_test_shuffle, y_test), callbacks=[earlystop])
ppd_cnn_model.save('ppd_cnn_model_noperm_'+DATASET_NAME+'.hdf5')


In [ ]:
SECRET_SEED = 87
#x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
#x_test_shuffle = permute_pixels(x_test, SECRET_SEED)
x_train_shuffle = x_train
x_test_shuffle = x_test

for idx in range(1,6):
    ppd_cnn_model = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=idx)
    ppd_cnn_model.summary()
    ppd_cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    ppd_cnn_model.fit(x_train_shuffle, y_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE, validation_data=(x_test_shuffle, y_test), 
                          callbacks=[earlystop, noNaN, ModelCheckpoint('ppd_cnn_model'+str(idx)+'_noperm_'+DATASET_NAME+'.hdf5', save_best_only=True)])
    


In [11]:
cnn_model = load_model('models/cnn_model_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model = load_model('models/ppd_cnn_model_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model1 = load_model('models/ppd_cnn_model1_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model2 = load_model('models/ppd_cnn_model2_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model3 = load_model('models/ppd_cnn_model3_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model4 = load_model('models/ppd_cnn_model4_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})
ppd_cnn_model5 = load_model('models/ppd_cnn_model5_noperm_'+DATASET_NAME+'.hdf5', custom_objects={'tf':tf})

In [9]:
## Get adv from CNN, CNN+PPD (87), CNN + Phase, CNN + Perm (87)
backend.set_learning_phase(0)
sess =  backend.get_session()
print(backend.learning_phase())
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

#x_adv_cnn = get_adv_exapmles(cnn_model, x_test, sess, attack_params)
#x_adv_ppd = get_adv_exapmles(ppd_cnn_model, x_test, sess, attack_params)

x_adv_ppd1 = get_adv_exapmles(ppd_cnn_model1, x_test, sess, attack_params)
#x_adv_ppd2 = get_adv_exapmles(ppd_cnn_model2, x_test, sess, attack_params)
#x_adv_ppd3 = get_adv_exapmles(ppd_cnn_model3, x_test, sess, attack_params)
#x_adv_ppd4 = get_adv_exapmles(ppd_cnn_model4, x_test, sess, attack_params)
#x_adv_ppd5 = get_adv_exapmles(ppd_cnn_model5, x_test, sess, attack_params)


0


In [12]:
## Test cnn_adv on cnn, test cnn+ppd (87) adv on cnn+ppd (87) and cnn+ppd (23)

def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)

# x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_shuffle = x_test

pred, acc = test_model(cnn_model, x_test, y_test)
print ('CNN acc normally', acc)
pred, acc = test_model(cnn_model, x_adv_cnn, y_test)
print ('CNN acc adv', acc)

# x_adv_shuffle = permute_pixels(x_adv_ppd, SECRET_SEED)
x_adv_shuffle = x_adv_ppd

pred, acc = test_model(ppd_cnn_model, x_shuffle, y_test)
print ('PPD CNN acc normally', acc)
pred, acc = test_model(ppd_cnn_model, x_adv_shuffle, y_test)
print ('PPD CNN acc adv', acc)



# ## hidden phase
x_adv = [x_adv_ppd1, x_adv_ppd2, x_adv_ppd3, x_adv_ppd4, x_adv_ppd5]
ppd_cnn_models = [ppd_cnn_model1, ppd_cnn_model2, ppd_cnn_model3, ppd_cnn_model4, ppd_cnn_model5]
for idx in range(1, 6):
    pred, acc = test_model(ppd_cnn_models[idx-1], x_shuffle, y_test)
    print ('PPD CNN phase_hidden = ', str(idx), ' acc ', acc)
    
    # x_adv_shuffle = permute_pixels(x_adv[idx-1], SECRET_SEED)
    x_adv_shuffle = x_adv[idx-1]
    pred, acc = test_model(ppd_cnn_models[idx-1], x_adv_shuffle, y_test)
    print ('PPD CNN phase_hidden = ', str(idx), ' acc adv', acc) 

CNN acc normally 0.793
CNN acc adv 0.136
PPD CNN acc normally 0.5536
PPD CNN acc adv 0.214
PPD CNN phase_hidden =  1  acc  0.1
PPD CNN phase_hidden =  1  acc adv 0.1
PPD CNN phase_hidden =  2  acc  0.6307
PPD CNN phase_hidden =  2  acc adv 0.0862
PPD CNN phase_hidden =  3  acc  0.1883
PPD CNN phase_hidden =  3  acc adv 0.1245
PPD CNN phase_hidden =  4  acc  0.5812
PPD CNN phase_hidden =  4  acc adv 0.0708
PPD CNN phase_hidden =  5  acc  0.7378
PPD CNN phase_hidden =  5  acc adv 0.13


In [ ]:
print(ppd_cnn_model1.predict(x_shuffle)[0])

In [ ]:
print(ppd_cnn_model1.layers[-13].get_weights())